<a href="https://colab.research.google.com/github/lagutinsy/Zadahi/blob/main/%D0%AD%D0%BA%D0%BE%D0%BD%D0%BE%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D0%BA%D0%B0%20%D0%94%D0%973.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ДЗ-3 Эконометрика весна 2024г Выполнил работу Лагутин С.Ю.

Для работы используем данные из статьи:

Bloom, N., Sadun, R. and Van Reenen, J. (2015). Do private equity owned firms have better management practices? American Economic Review, 105(5), pp.442-446

Файл загружен из репозитария

https://www.openicpsr.org/openicpsr/project/113354/version/V1/view?path=/openicpsr/113354/fcr:versions/V1/PE/petopost.dta&type=file

Задание

А) Воспроизведите таблицу 2 с помощью регрессии с кросс-секционными данными.

Б) оцените аналог таблицы 2 в рамках регрессии с панельными данными (фирма единица наблюдения, выделите наблюдения для каждой фирмы во времени), оцените модель пулд-данных, а также модели со случайными или фиксированными эффектами и объясните, какая предпочтительнее из экономических соображений.


In [47]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.stats.diagnostic import linear_harvey_collier
from linearmodels.iv import IV2SLS, compare
from linearmodels.shared.hypotheses import WaldTestStatistic as wald_test


import linearmodels.iv as iv
import statsmodels.formula.api as smf
import pandas as pd
from linearmodels.panel import PooledOLS, BetweenOLS, RandomEffects, PanelOLS

Загружаем данные в систему и пвоспроизводим таблицу, с промотром данных для выполнения аналитики.

In [48]:
data_not_panel = pd.read_stata('/content/petopost.dta')
data_not_panel.head()

,cty,country,analyst,i_comptenure,emp_firm,emp_plant,ownership,duration,i_seniority,year,...,o_famextceo,o_dispersed,o_pe,o_private,o_gov,o_other,o_managers,zorg_inputs,zorg_demand,account_id
0,us,United States,Jayesh Patel,5,400.0,NaN,Dispersed Shareholders,46.0,4,2004,...,0,1,0,0,0,0,0,NaN,NaN,1
1,us,United States,May Yoon,6,600.0,320.0,Dispersed Shareholders,60.0,3,2006,...,0,1,0,0,0,0,0,0.936675,-0.747304,1
2,us,United States,Simone Bohnenberger-Rich,10,2000.0,60.0,Dispersed Shareholders,55.0,3,2010,...,0,1,0,0,0,0,0,1.123721,0.706634,1
3,us,United States,PG,10,3100.0,NaN,Founder,45.0,3,2002,...,0,0,0,0,0,0,0,NaN,NaN,2
4,us,United States,Michael Bevan,10,189.0,NaN,Dispersed Shareholders,40.0,4,2004,...,0,1,0,0,0,0,0,NaN,NaN,3


In [49]:

ind_var_o = ["o_founder", "o_famceo", "o_famextceo", "o_dispersed", "o_private", "o_gov", "o_other", "o_managers"]
# Определение независимых переменных
formula = "zmanagement ~ " + " + ".join(ind_var_o)
# Формулу для линейной регрессии
model1 = sm.OLS.from_formula(formula, data=data_not_panel).fit()
# Модель линейной регрессии
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:            zmanagement   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     287.9
Date:                Thu, 25 Apr 2024   Prob (F-statistic):               0.00
Time:                        07:52:23   Log-Likelihood:                -20262.
No. Observations:               15038   AIC:                         4.054e+04
Df Residuals:                   15029   BIC:                         4.061e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.3743      0.043      8.668      

Записи:
[1] Стандартные ошибки предполагают, что ковариационная матрица ошибок задана правильно.

Данные схожи с результатами  статьи.


In [50]:
data_pan = data_not_panel.set_index(['account_id', 'year'])
data_pan.head()

cty        country                   analyst  i_comptenure  \
account_id year                                                              
1          2004  us  United States              Jayesh Patel             5   
           2006  us  United States                  May Yoon             6   
           2010  us  United States  Simone Bohnenberger-Rich            10   
2          2002  us  United States                        PG            10   
3          2004  us  United States             Michael Bevan            10   

                 emp_firm  emp_plant               ownership  duration  \
account_id year                                                          
1          2004     400.0        NaN  Dispersed Shareholders      46.0   
           2006     600.0      320.0  Dispersed Shareholders      60.0   
           2010    2000.0       60.0  Dispersed Shareholders      55.0   
2          2002    3100.0        NaN                 Founder      45.0   
3          2004     189.0        NaN  Dispersed Shareholders      40.0   

                 i_seniority  reliability  ...  o_famceo  o_famextceo  \
account_id year                            ...                          
1          2004            4          -99  ...         0            0   
           2006            3           10  ...         0            0   
           2010            3            8  ...         0            0   
2          2002            3          -99  ...         0            0   
3          2004            4          -99  ...         0            0   

                 o_dispersed  o_pe  o_private  o_gov  o_other  o_managers  \
account_id year                                                             
1          2004            1     0          0      0        0           0   
           2006            1     0          0      0        0           0   
           2010            1     0          0      0        0           0   
2          2002            0     0          0      0        0           0   
3          2004            1     0          0      0        0           0   

                 zorg_inputs  zorg_demand  
account_id year                            
1          2004          NaN          NaN  
           2006     0.936675    -0.747304  
           2010     1.123721     0.706634  
2          2002          NaN          NaN  
3          2004          NaN          NaN  

[5 rows x 39 columns]

In [51]:
data_pan['cty'] = data_pan['cty'].astype('category')
data_pan['sic_survey'] = data_pan['sic_survey'].astype('category')

model2 = PanelOLS.from_formula("zmanagement ~ " + " + ".join(ind_var_o) + ' + cty + sic_survey', data=data_pan).fit()
print(model2)

                          PanelOLS Estimation Summary                           
Dep. Variable:            zmanagement   R-squared:                        0.2679
Estimator:                   PanelOLS   R-squared (Between):              0.2933
No. Observations:               15038   R-squared (Within):              -0.0247
Date:                Thu, Apr 25 2024   R-squared (Overall):              0.2679
Time:                        07:52:44   Log-likelihood                -1.899e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      27.426
Entities:                       11370   P-value                           0.0000
Avg Obs:                       1.3226   Distribution:               F(198,14839)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             27.434
                            

In [52]:
ownership = ["o_founder", "o_famceo", "o_famextceo", "o_dispersed", "o_private", "o_gov", "o_other", "o_managers"]
firm_a = ["lemp_firm", "lemp_firmm"]
firm_b = ["lfirmage", "lfirmagem"]
firm_c = ["ldegree_t", "ldegree_tm"]
noise = ["analyst", "reliability", "reliability_miss", "i_comptenure", "i_comptenure_miss",
         "i_seniority", "i_seniority_miss", "duration", "duration_miss"]


In [53]:
formula3 = "zmanagement ~ " + " + ".join(ownership) + " + " + " + ".join(noise) + " + " + " + ".join(firm_a) + " + " + " + ".join(firm_b) + " + " + " + ".join(firm_c) + ' + cty + sic_survey'

model3 = PanelOLS.from_formula(formula3, data=data_pan, check_rank=False).fit()
print(model3)

/usr/local/lib/python3.10/dist-packages/linearmodels/panel/results.py:88: RuntimeWarning: invalid value encountered in sqrt
  return Series(np.sqrt(np.diag(self.cov)), self._var_names, name="std_error")


                          PanelOLS Estimation Summary                           
Dep. Variable:            zmanagement   R-squared:                        0.4654
Estimator:                   PanelOLS   R-squared (Between):              0.5017
No. Observations:               15038   R-squared (Within):               0.0808
Date:                Thu, Apr 25 2024   R-squared (Overall):              0.4654
Time:                        07:53:19   Log-likelihood                -1.663e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      28.410
Entities:                       11370   P-value                           0.0000
Avg Obs:                       1.3226   Distribution:               F(447,14590)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             21.072
                            

In [54]:
model4 = PanelOLS.from_formula("zmanagement ~ " + " + ".join(ind_var_o) + ' + cty + sic_survey + EntityEffects', data=data_pan, drop_absorbed=True).fit()
print(model4)

<ipython-input-54-0d74b1a4ba9e>:1: AbsorbingEffectWarning: 
Variables have been fully absorbed and have removed from the regression:

cty[T.ar], cty[T.au], cty[T.br], cty[T.ca], cty[T.cn], cty[T.co], cty[T.es], cty[T.et], cty[T.fr], cty[T.gb], cty[T.ge], cty[T.gh], cty[T.gr], cty[T.in], cty[T.ir], cty[T.it], cty[T.jp], cty[T.ke], cty[T.mm], cty[T.mx], cty[T.mz], cty[T.nc], cty[T.ng], cty[T.nz], cty[T.po], cty[T.pt], cty[T.sg], cty[T.sw], cty[T.tr], cty[T.tz], cty[T.us], cty[T.vn], cty[T.zm], sic_survey[T.201], sic_survey[T.202], sic_survey[T.203], sic_survey[T.204], sic_survey[T.205], sic_survey[T.206], sic_survey[T.207], sic_survey[T.208], sic_survey[T.209], sic_survey[T.210], sic_survey[T.211], sic_survey[T.212], sic_survey[T.219], sic_survey[T.220], sic_survey[T.221], sic_survey[T.222], sic_survey[T.223], sic_survey[T.224], sic_survey[T.225], sic_survey[T.226], sic_survey[T.227], sic_survey[T.228], sic_survey[T.229], sic_survey[T.230], sic_survey[T.231], sic_survey[T.232], sic_surve

                          PanelOLS Estimation Summary                           
Dep. Variable:            zmanagement   R-squared:                        0.0058
Estimator:                   PanelOLS   R-squared (Between):             -1.0079
No. Observations:               15038   R-squared (Within):               0.0058
Date:                Thu, Apr 25 2024   R-squared (Overall):             -0.6995
Time:                        07:53:40   Log-likelihood                   -4661.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2.3708
Entities:                       11370   P-value                           0.0114
Avg Obs:                       1.3226   Distribution:                  F(9,3659)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             2.3708
                            

F-тест на возможность объединения: 2,2491

P-значение: 0,0000

Распределение: F(11369,3659)


Включенные эффекты: Сущность


In [56]:
model5 = RandomEffects.from_formula("zmanagement ~ " + " + ".join(ind_var_o) + ' + cty + sic_survey', data=data_pan).fit()
print(model5) # Случайные эффекты

                        RandomEffects Estimation Summary                        
Dep. Variable:            zmanagement   R-squared:                        0.2427
Estimator:              RandomEffects   R-squared (Between):              0.2943
No. Observations:               15038   R-squared (Within):              -0.0202
Date:                Thu, Apr 25 2024   R-squared (Overall):              0.2671
Time:                        07:54:36   Log-likelihood                -1.559e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      24.015
Entities:                       11370   P-value                           0.0000
Avg Obs:                       1.3226   Distribution:               F(198,14839)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             24.075
                            

In [58]:
model6 = PanelOLS.from_formula(formula + ' + EntityEffects', data=data_pan).fit()
# Фике эффекты 2
print(model6)


                          PanelOLS Estimation Summary                           
Dep. Variable:            zmanagement   R-squared:                        0.0058
Estimator:                   PanelOLS   R-squared (Between):              0.0360
No. Observations:               15038   R-squared (Within):               0.0058
Date:                Thu, Apr 25 2024   R-squared (Overall):              0.0321
Time:                        07:55:32   Log-likelihood                   -4661.3
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2.6679
Entities:                       11370   P-value                           0.0064
Avg Obs:                       1.3226   Distribution:                  F(8,3660)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             2.6679
                            

F-тест на возможность объединения: 2,2416

P-значение: 0,0000

Распределение: F(11369,3660)


In [59]:
model7 = RandomEffects.from_formula(formula, data=data_pan).fit()
#случайные эффекты-2
print(model7)

                        RandomEffects Estimation Summary                        
Dep. Variable:            zmanagement   R-squared:                        0.1053
Estimator:              RandomEffects   R-squared (Between):              0.1466
No. Observations:               15038   R-squared (Within):              -0.0461
Date:                Thu, Apr 25 2024   R-squared (Overall):              0.1265
Time:                        07:56:19   Log-likelihood                 -1.54e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      221.19
Entities:                       11370   P-value                           0.0000
Avg Obs:                       1.3226   Distribution:                 F(8,15030)
Min Obs:                       1.0000                                           
Max Obs:                       5.0000   F-statistic (robust):             221.19
                            

Коэффициенты оказались не имеющими значения, что делает невозможным применение модели с фиксированными эффектами. Как уже отмечалось ранее, нарушение предположений о гомоскедастичности и отсутствии автокорреляции делает неприменимой модель с пулданными. Поэтому, единственным вариантом, оставшимся после исключения других методов, является использование модели со случайными эффектами.
